In [31]:
from importLib import import_from_github, import_from_github_using_path
# imported tutorial
import_from_github(r"https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/extras/exercises/03_pytorch_computer_vision_exercises.ipynb")
import_from_github_using_path("/ml_toolkit\ml_funcs.py")
# libraries
import torch as pt
print(f"PyTorch version: {pt.__version__}")

device = 'cuda' if pt.cuda.is_available() else 'cpu'



c:\Users\22gusgam\Desktop\k\PyTorch\03\exercises/03_pytorch_computer_vision_exercises.ipynb already exists
c:\Users\22gusgam\Desktop\k\PyTorch\03\exercises/ml_toolkit\ml_funcs.py doesn't exist, download


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\22gusgam\\Desktop\\k\\PyTorch\\03\\exercises/ml_toolkit\\ml_funcs.py'

##### QNA

What are 3 areas in industry where computer vision is currently being used?
1. Tesla autopilot
2. Face recogition
3. herbs and plants recognition (i.e telling which the name of plants from photos)

Search "what is overfitting in machine learning" and write down a sentence about what you find. 
* It is when the module is to fitted on the current data and therefore doesn't react and produce reliable results on new data

Search "ways to prevent overfitting in machine learning", write down 3 of the things you find and a sentence about each
1. Decrease the Model Complexity
2. Data Augmentation
    - Technique of artificially increasing the training set by creating modified copies of a dataset using existing data
3. Weight Regularization
    - Technique in which slight modifications are made to the learning algorithm which in its turn makes the model generalize better

### Load dataset Fer2013

In [2]:
import torchvision as tv
from torchvision.transforms import ToTensor
import os
train_data = tv.datasets.MNIST(root=f"{os.getcwd()}/data",train=True, transform=ToTensor, target_transform=False,download=True)

test_data = tv.datasets.MNIST(root=f"{os.getcwd()}/data",train=False, transform=ToTensor, target_transform=False,download=True)


In [16]:
print(len(train_data), len(test_data))

60000 10000
tensor([5, 0, 4,  ..., 5, 6, 8])


### Visualize data

In [24]:
from mlfuncs import 
print(train_data.data[:1])

print(train_data.targets[:1])

tensor([[[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,
           18,  18, 126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,